# 0.

1. 只有1个GT模型input

In [50]:
%load_ext autoreload
%autoreload 2

# 1. Preparations
## 1.1 Import

In [51]:
import pandas as pd
import numpy as np
import math
from importlib import reload
import random
import torch.utils.data
from mydataset import *
import torch.nn as nn
from torch.utils.data import DataLoader, SubsetRandomSampler
import torch.nn.functional as F
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
# from tensorboardX import SummaryWriter
from visdom import Visdom
from graphviz import Digraph
from torchviz import make_dot
from torch.nn.utils.rnn import pad_sequence
import pygraphviz as pgv
import networkx as nx
import matplotlib.pyplot as plt
import geomloss
import time

import scipy
from torch.nn import KLDivLoss
from torch.autograd.gradcheck import gradcheck
from config import bcolors

import config
import loss
import plot

## 1.2 Global settings and reloading

In [52]:
reload(config)  # 必须reload！！
reload(loss)    # 必须reload！！
reload(plot)
from config import DefaultConfig
from loss import cal_metric
from loss import loss_fn_v2
from loss import validate
from loss import loss_fn_WD
from plot import plot_conv_weight
from plot import plot_mu_weight
from plot import plot_pi_weight
from plot import plot_sigma_weight
from plot import plot_net

opt = DefaultConfig()


# nums of Gaussian kernels
N_gaussians = opt.N_gaussians

total_train_step = 0
total_test_step = 0

# Range【一般用不到】
a = 0
b = 255


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


# 2. Dataset and Dataloader
1. DataLoader中的`shuffer=True`表示在每一次epoch中都打乱所有数据的顺序，然后以batch为单位从头到尾按顺序取用数据。这样的结果就是不同epoch中的数据都是乱序的,设置随机种子的作用就是让你的每一次训练都乱的一样，

## 2.1 Dataset and spliting


- 设置随机数种子

In [53]:
def setup_seed(seed):
    torch.manual_seed(seed)
    # torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    # torch.backends.cudnn.deterministic = True

setup_seed(7)

In [54]:
dataset = myDataset(opt.train_path, opt.target_path_metric, opt.target_path_loss, opt.data_key_path, opt.NLL_metric_path)

In [55]:
shuffled_indices = []
DATA_len = 0
# 使用全部的data
if not opt.arr_flag:
    DATA_len = dataset.__len__()
    shuffled_indices = np.random.permutation(DATA_len)

# 使用指定的data
if opt.arr_flag:
    shuffled_indices = np.load(opt.arr_path)
    DATA_len = len(shuffled_indices)
    np.random.shuffle(shuffled_indices)

train_idx = shuffled_indices[:int(opt.train_pct*DATA_len)]
# train_idx = shuffled_indices
tmp = int((opt.train_pct+opt.vali_pct)*DATA_len)
val_idx = shuffled_indices[int(opt.train_pct*DATA_len):tmp]

test_idx = shuffled_indices[tmp:]
print(train_idx)

[ 807  305  455  939  508  594  835 1082  598 1102   46  761  841  141
  407  334  253  500  734  936  698  446  907 1087 1009  140  463  547
 1155  856   34 1156  703 1121  751  587 1100  509  473  128  788   97
  471  385 1085  525  679 1135  284 1146  186  318 1088 1113    9  457
 1047  451  569 1120 1074  326  377  809  109   98  620 1194  825  828
   83  113  556  674  568  853  351  558   54  656  804 1149  101  344
  851  544  955   40 1021  489  626  664  657  868 1169  151  179 1130
  564  713  743 1014  966  861 1132  146  610  408  662  551  172 1020
  982 1148  431  517  270  858  170  374  816  618  205   17   53 1003
  263  857  716  843  498  228  339  725  752  278  649 1017  108  642
 1195 1174   99  530  632  888  189  961  358 1078  663  757 1051  204
  409  283  562   23  619  216  474  921  950 1162  123  785  769  621
  262  586 1178  541  795   70 1189  396  171  845  168 1125  361  224
  125  706  164  231  264   42  746  872  998  132 1035  430   38  522
  880 

## 2.2 Dataloader and collating
1. 主要是对label数据进行collate
    - 按照batch中的最大target data长度进行padding，padding with 0
2. 返回的结果多一个batch dim,比如下面的`5`
    - After collating:
        - `torch.Size([5, 3, 300]),torch.Size([5, 87, 2])`
        - `87`是最长的targets data长度

In [56]:
def my_collate_fn(data):
# 这里的data是一个list， list的元素是元组: (self.data, self.label)
# collate_fn的作用是把[(data, label),(data, label)...]转化成([data, data...],[label,label...])
# 假设self.data的一个data的shape为(channels, length), 每一个channel的length相等,
# data[索引到index(batch)][索引到data或者label][索引到channel]

    data_list = []
    target_metric_list = []
    target_loss_list = []
    setting_list = []
    metric_list = []

    data_len = len(data)      # 读进来的data batch的大小

    batch = 0

    while batch < data_len:
        # print("shape:",data[batch][0].shape) #shape: (3, 300)

        data_tmp = np.array([data[batch][0][opt.GT_CHOSEN,:],data[batch][0][2,:]])
        data_list.append(torch.tensor(data_tmp))

        target_metric_list.append(torch.tensor(data[batch][1]))
        target_loss_list.append(torch.tensor(data[batch][2]))
        setting_list.append(torch.tensor(data[batch][3]))
        metric_list.append(torch.tensor(data[batch][4]))
        batch += 1

    # Pad target data with zeros
    target_metric_padded = pad_sequence(target_metric_list,batch_first=True)
    target_loss_padded = pad_sequence(target_loss_list,batch_first=True)
    target_metric_tensor = target_metric_padded.float()
    target_loss_padded = target_loss_padded.float()

    data_tensor = torch.stack(data_list).float()
    setting_tensor = torch.stack(setting_list).float()
    metric_tensor = torch.stack(metric_list).float()

    return data_tensor, target_metric_tensor, target_loss_padded, setting_tensor, metric_tensor

In [57]:
def my_collate_fn_2(data):
# 这里的data是一个list， list的元素是元组: (self.data, self.label)
# collate_fn的作用是把[(data, label),(data, label)...]转化成([data, data...],[label,label...])
# 假设self.data的一个data的shape为(channels, length), 每一个channel的length相等,
# data[索引到index(batch)][索引到data或者label][索引到channel]

    data_list = []              # training data
    target_metric_list = []     # target data for computing metric of NN, (TARGET=1)
    target_loss_list = []       # target data for computing loss of NN, (TARGET=5)
    setting_list = []
    metric_list = []

    data_len = len(data)        # 读进来的data batch的大小

    batch = 0

    while batch < data_len:
        # print("shape:",data[batch][0].shape) #shape: (3, 300)

        # 对于所有用0填充的data来说，最好用一个数字补齐这些0
        data[batch][0][opt.GT_CHOSEN,np.where(data[batch][0][opt.GT_CHOSEN]==0)]= min(min(data[batch][0][opt.GT_CHOSEN]),opt.SAFETY)
        # 然后再拼接
        data_tmp = np.array([data[batch][0][opt.GT_CHOSEN,:],data[batch][0][2,:]])
        data_list.append(torch.tensor(data_tmp))

        data_list.append(torch.tensor(data[batch][0]))
        target_metric_list.append(torch.tensor(data[batch][1]))
        target_loss_list.append(torch.tensor(data[batch][2]))
        setting_list.append(torch.tensor(data[batch][3]))
        metric_list.append(torch.tensor(data[batch][4]))
        batch += 1

    # Pad target data with zeros
    target_metric_padded = pad_sequence(target_metric_list,batch_first=True)
    target_loss_padded = pad_sequence(target_loss_list,batch_first=True)
    target_metric_tensor = target_metric_padded.float()
    target_loss_padded = target_loss_padded.float()

    data_tensor = torch.stack(data_list).float()
    setting_tensor = torch.stack(setting_list).float()
    metric_tensor = torch.stack(metric_list).float()

    return data_tensor, target_metric_tensor, target_loss_padded, setting_tensor, metric_tensor

In [58]:
train_loader = DataLoader(dataset = dataset,batch_size = opt.batch_size, shuffle=False, num_workers=0, drop_last=False, sampler=SubsetRandomSampler(train_idx), collate_fn = my_collate_fn)

val_loader = DataLoader(dataset = dataset,batch_size = opt.batch_size, shuffle=False, num_workers=0, drop_last=False, sampler=SubsetRandomSampler(val_idx),collate_fn = my_collate_fn)

# 注意test_loader的batch size
test_loader = DataLoader(dataset = dataset,batch_size = opt.batch_size, shuffle=False, num_workers=0, drop_last=False, sampler=SubsetRandomSampler(test_idx),collate_fn = my_collate_fn)

# 3. The Net and Init
1. BatchNorm1d: The mean and std are calculated per-dimension over the mini-batches
## 3.1 2层MLP

In [59]:
# 参数量92,000+
# 2层
class MLP_1(nn.Module):
    # code->generate->override methods
    def __init__(self, n_gaussians) -> None:
        super().__init__()
        self.BN1 = nn.BatchNorm1d(num_features=2,affine=True)
        self.BN2 = nn.BatchNorm1d(num_features=150,affine=True)
        self.BN3 = nn.BatchNorm1d(num_features=12,affine=True)
        self.drop = nn.Dropout(0.3)

        self.linear1 = nn.Linear(600, 150)
        self.linear2 = nn.Linear(150, 12)

        self.ac_func = nn.Softplus()

        self.flatten = nn.Flatten()

        self.z_pi = nn.Sequential(
            nn.Linear(12, n_gaussians),  # 30个params要learn
            nn.Softmax(dim=1)
        )

        self.z_mu = nn.Linear(12, n_gaussians)
        self.z_sigma = nn.Linear(12, n_gaussians)

        # SELU Init
        # for m in self.modules():
        #     if isinstance(m, (nn.Linear)):
        #         nn.init.kaiming_normal_(m.weight, nonlinearity='linear')

    def forward(self, x):
        x = self.BN1(x)
        x = self.flatten(x)

        x = self.linear1(x)
        x = self.BN2(x)
        x = self.ac_func(self.drop(x))

        x = self.linear2(x)
        x = self.BN3(x)
        x = self.ac_func(self.drop(x))

        pi = self.z_pi(x)
        # pi = self.z_pi(torch.clamp(x,1e-3))
        mu = self.z_mu(x)
        # sigma = torch.exp(self.z_sigma(x))
        sigma = F.elu(self.z_sigma(x)) + 1
        sigma = torch.clamp(sigma,1e-4)

        # return x
        return pi, mu, sigma

## 3.2 Conv

In [60]:
# Not Sequential
# 573, 588, 693
class MLP_1_1(nn.Module):
    # code->generate->override methods
    def __init__(self, n_gaussians) -> None:
        super().__init__()

        self.kernel_size = (2,15)
        # (3,15),19
        self.stride = (2,8)
        self.ln_in = int((300-self.kernel_size[1])/self.stride[1]+1)

        self.BN1 = nn.BatchNorm1d(num_features=2,affine=True)

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=self.kernel_size, stride=self.stride, padding=0,bias=True)
        self.conv2 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=self.kernel_size, stride=self.stride, padding=0,bias=True)
        self.conv3 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=self.kernel_size, stride=self.stride, padding=0,bias=True)

        self.BN_aff1 = nn.BatchNorm1d(num_features=self.ln_in,affine=True)
        self.BN_aff2 = nn.BatchNorm1d(num_features=self.ln_in,affine=True)
        self.BN_aff3 = nn.BatchNorm1d(num_features=self.ln_in,affine=True)

        self.ac_func = nn.Softplus()

        self.z_pi = nn.Sequential(
            nn.Linear(self.ln_in, n_gaussians),
            nn.Softmax(dim=1)
        )

        self.z_mu = nn.Linear(self.ln_in, n_gaussians)
        self.z_sigma = nn.Linear(self.ln_in, n_gaussians)

    def forward(self, x):

        x = self.BN1(x)
        x = torch.unsqueeze(x,dim=1)                     # torch.Size([B, 1, 3, 300])

        x1 = F.softplus(self.conv1(x))                   # [B, 1, 1, ln_in]
        x2 = F.softplus(self.conv2(x))                   # [B, 1, 1, ln_in]
        x3 = F.softplus(self.conv3(x))                   # [B, 1, 1, ln_in]

        x1_1 = torch.squeeze(x1)
        x2_1 = torch.squeeze(x2)
        x3_1 = torch.squeeze(x3)

        x1_2 = self.BN_aff1(x1_1)
        x2_2 = self.BN_aff2(x2_1)
        x3_2 = self.BN_aff3(x3_1)

        x1_3 = F.softplus(x1_2)
        x2_3 = F.softplus(x2_2)
        x3_3 = F.softplus(x3_2)

        pi = self.z_pi(x1_3)
        mu = self.z_mu(x2_3)

        sigma = torch.exp(self.z_sigma(x3_3))
        sigma = torch.clamp(sigma,1e-4)

        # return x1_3
        return pi, mu, sigma

# 4. The Loss

## 4.0 The metric
1. 以NLL作为metric
2. 理论模型的metric可以提前算好，读进来，这里读setting是为了？IDK

In [61]:
# from loss import cal_metric

## 4.1 NLL loss

In [62]:
# from loss import loss_fn_v2

## 4.4 W-distance
1. 拿到target data的两列，按照第一列去predict

In [63]:
# from loss import loss_fn_WD

# 5 Plot
1. draw:
    - loss: training data的loss和test data的loss趋势
    - MLP的网络结构（.png）
    - target distrb.和pred. distrb.
    - 所有target data的分布图


In [64]:
writer = SummaryWriter("logs-MLP")
viz = Visdom(env="001")
print("Done")

Setting up a new session...


Done


## 5.2 plot mdn
### 5.2.1 mdn pdf的图像[with input data]
1. 求解[1,6,11...]上的cdf：
    - 求解[1,6,11..]的cdf，再求解[6,11,..]，两者相减

In [65]:
def draw_mdn_2(pi, mu, sigma, target, input, total_train_step, N_gaussians):
    """
    画两条曲线，pred-1表示归一化前，pred-2表示归一化后

    :param pi:
    :param mu:
    :param sigma:
    :param target:
    :param input:
    :param total_train_step:
    :param N_gaussians:
    :return:
    """
    # 只画一个batch中的第一个

    # The target distrb.
    target = torch.unique(target,dim=0)    # drop the duplicate
    n_target = target[:,0]
    non_zero_idx = torch.nonzero(n_target)
    n = n_target[non_zero_idx]

    p_target = target[:,1]
    p = p_target[non_zero_idx]
    max_n = int(max(n).item())

    # The predicted distrb.
    m = torch.distributions.Normal(mu,sigma)          # Gaussian
    # m = torch.distributions.Laplace(mu,sigma)           # Laplace

    x_0 = torch.arange(1,(max_n+TARGET),TARGET).to(device=device)

    x = torch.repeat_interleave(x_0.unsqueeze(dim=1), repeats=N_gaussians, dim=1)   # expand dim
    # 求解每个长度为TARGET的区间上的cdf
    y = (m.cdf(x+TARGET) - m.cdf(x)).to(device=device)

    # 方法一：
    y_pred_1 = torch.sum(pi*y,dim=1)
    # 方法二：做一下(0,1)上的归一化
    y_pred_2 = y_pred_1/y_pred_1.sum()

    # The input distrb.
    # input_data = input[0:2,:]
    input_data = input[0,:]
    x_input = torch.arange(1,(300+1),SCALE).to(device=device)
    y_input_0 = input_data.to(device=device)
    # y_input_0 = (input_data[0,:]).to(device=device)
    # y_input_1 = (input_data[1,:]).to(device=device)

    # Init
    win_str = "total_train_step = "+str(total_train_step)
    title_str = "Distrb. in "+win_str
    viz.line(X = [0.],Y = [0.], env="001", win=win_str, opts= dict(title=title_str))

    # Visdom本身不能把hist和line画在一个window中
    # 如果想画一起只能是两条line
    # Plot y_target
    # viz.histogram(X = n, env="001", win=win_str,
    #         opts= dict(title=title_str))
    viz.line(X = n,Y= p, env="001", win=win_str, update="append", name='target',
            opts= dict(title=title_str,markers = True,markersize = 7,markersymbol = "cross-thin-open"))

    # Plot y_pred
    viz.line(X = x_0,Y= y_pred_1, env="001", win=win_str, update="append", name='pred-1',
            opts= dict(title=title_str))

    # Plot y_pred_2
    viz.line(X = x_0,Y= y_pred_2, env="001", win=win_str, update="append", name='pred-2',
            opts= dict(title=title_str))

    # Plot y_input
    # 如果input data长度短，全部画完
    if(len(x_input) <= max_n):
        viz.line(X = x_input,Y= y_input_0, env="001", win=win_str, update="append", name='GT-1', opts= dict(title=title_str))
        # viz.line(X = x_input,Y= y_input_1, env="001", win=win_str, update="append", name='GT-2', opts= dict(title=title_str))
    # 如果input data长度长，则截断
    else:
        x_input_0 = x_input[0:max_n]
        y_input_2 = y_input_0[0:max_n]/y_input_0[0:max_n].sum()
        # y_input_3 = y_input_1[0:max_n]/y_input_1[0:max_n].sum()
        viz.line(X = x_input_0,Y= y_input_2, env="001", win=win_str, update="append", name='GT-1', opts= dict(title=title_str))
        # viz.line(X = x_input_0,Y= y_input_3, env="001", win=win_str, update="append", name='GT-2', opts= dict(title=title_str))

## 5.4 plot the loss
1. 直接根据参数决定draw什么loss，反正loss都画在一个window里面

In [66]:
win_train_loss_str = "The Loss of BATCH in the Training Data"
win_vali_loss_str = "The Loss in the Vali Data"
win_train_epoch_loss_str = "The Loss of EPOCH in the Training Data"

def draw_loss(X_step, loss, win_str):
    viz.line(X = [X_step], Y = [loss],win=win_str, update="append",
        opts= dict(title=win_str))


## 5.5 plot the metric

In [67]:
win_vali_metric_str = "The NLL of ALL vali data"

def draw_metric(X_step, total_vali_metric, GT_metric):

    viz.line(X = [X_step], Y = [[total_vali_metric,GT_metric[0,0],GT_metric[0,1]]],win=win_vali_metric_str, update="append", opts= dict(title=win_vali_metric_str, legend=['pred','GT-1','GT-2'], showlegend=True,xlabel="epoch", ylabel="NLL"))

# 6 Training
## 6.1 Init

In [68]:
# torch.autograd.set_detect_anomaly(True)

mlp = MLP_1_1(N_gaussians)

# Init the params
# # mlp = model_param_init(mlp)
# Init the vis
viz.line(X = [0.],Y = [0.], env="001", win=win_train_loss_str, opts= dict(title=win_train_loss_str))
# viz.line(X = [0.],Y = [0.], env="001", win=win_vali_loss_str, opts= dict(title=win_vali_loss_str))
viz.line(X = [0.],Y = [0.], env="001", win=win_train_epoch_loss_str, opts= dict(title=win_train_epoch_loss_str))
viz.line(X = [0.],Y = [[0.,0.,0.]], env="001", win=win_vali_metric_str, opts= dict(title=win_vali_metric_str,legend=['pred','GT-1','GT-2'], showlegend=True,xlabel="epoch", ylabel="NLL"))
# Save the init params
# torch.save(mlp.state_dict(), 'mlp_init.pth')

# Load the saved model
# model_path_WD = "mlp_train=300_WD.pth"
# model_path_LSE = "mlp_train=300_LSE.pth"
# model_data = torch.load(model_path_LSE)
# mlp.load_state_dict(model_data)

mlp = mlp.to(device=device)
summary(mlp, (2,300))

############ learning rate strategy ############
# 1.
# Set different lr for params
# id: id() 函数返回对象的唯一标识符，标识符是一个整数。返回对象的内存地址。
mu_params = list(map(id, mlp.z_mu.parameters()))
sigma_params = list(map(id, mlp.z_sigma.parameters()))
pi_params = list(map(id, mlp.z_pi.parameters()))
conv2_params = list(map(id, mlp.conv2.parameters()))

params_id = mu_params + sigma_params + pi_params + conv2_params

base_params = filter(lambda p: id(p) not in params_id, mlp.parameters())
params = [{'params': base_params},         # 如果对某个参数不指定学习率，就使用最外层的默认学习率
          {'params': mlp.z_pi.parameters(), 'lr': opt.lr_for_pi},
        {'params': mlp.z_mu.parameters(), 'lr': opt.lr_for_mu},
        {'params': mlp.conv2.parameters(), 'lr': opt.lr_for_mu},
          {'params': mlp.z_sigma.parameters(), 'lr': opt.lr_for_sigma}]

optimizer = torch.optim.Adam(params, lr=opt.learning_rate,weight_decay=opt.weight_decay)

# 2.
# Or set lr decay
# StepLR为步进，每step_size个epoch，lr*gamma
# scheduler  = torch.optim.lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.8)
# 下面这个比较好用
scheduler  = torch.optim.lr_scheduler.StepLR(optimizer,step_size=opt.StepLR_step_size,gamma=opt.StepLR_gamma)

# # Set the hooks
# #mlp.conv1.register_forward_hook(hook_forward_fn)
# mlp.conv1.register_full_backward_hook(hook_backward_fn)
#
# #mlp.linear1.register_forward_hook(hook_forward_fn)
# #mlp.linear1.register_full_backward_hook(hook_backward_fn)
#
# #mlp.z_pi.register_forward_hook(hook_forward_fn)
# mlp.z_pi.register_full_backward_hook(hook_backward_fn_pi)
#
# #mlp.z_mu.register_forward_hook(hook_forward_fn)
# mlp.z_mu.register_full_backward_hook(hook_backward_fn_mu)
#
# mlp.z_sigma.register_forward_hook(hook_forward_fn)
# mlp.z_sigma.register_full_[2, 3, 300]backward_hook(hook_backward_fn_sigma)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       BatchNorm1d-1               [-1, 2, 300]               4
            Conv2d-2             [-1, 1, 1, 36]              31
            Conv2d-3             [-1, 1, 1, 36]              31
            Conv2d-4             [-1, 1, 1, 36]              31
       BatchNorm1d-5                   [-1, 36]              72
       BatchNorm1d-6                   [-1, 36]              72
       BatchNorm1d-7                   [-1, 36]              72
            Linear-8                    [-1, 3]             111
           Softmax-9                    [-1, 3]               0
           Linear-10                    [-1, 3]             111
           Linear-11                    [-1, 3]             111
Total params: 646
Trainable params: 646
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward

## 6.3 START HERE
1. 使用Laplace分布改：
    - loss function和draw

In [69]:
# filename = "../log_file.txt"
# f = open(filename,'w')
total_train_step = 0
EPOCH_NUM = 60

train_start_time = time.time()
mlp.train()

for epoch in range(EPOCH_NUM):
    # # Save the net structure
    # net_file_name = "net_before_epoch"+str(epoch)+".pth"
    # net_path = net_root_path + net_file_name
    # torch.save(mlp.state_dict(), net_path)

    epoch_train_loss = 0
    print(f"========== Now this is EPOCH {epoch} ==========")
    for batch_id,data in enumerate(train_loader):

        input_data, _, target_loss, setting, _ = data
        # print(f"---- {batch_id} batch----")
        # Do the inference
        input_data = input_data.to(device)
        target_loss = target_loss.to(device)
        pi, mu, sigma = mlp(input_data)

        # Save the params
        # params = list(mlp.named_parameters())

        # Cal the MLE loss and draw the distrb.
        loss = loss_fn_v2(pi, mu, sigma, target_loss, opt.N_gaussians,opt.TARGET, opt.SAFETY, device)
        # loss = loss_fn_v3(pi, mu, sigma, target, N_gaussians)
        # loss = loss_fn_v5(pi, mu, sigma, target, N_gaussians)

        # Cal the CDF loss and draw the distrb.
        # loss = loss_fn_cdf(pi, mu, sigma, target, N_gaussians)

        # Cal the CE or KL loss and draw the distrb.
        # loss  = loss_fn_CE(pi, mu, sigma, target, N_gaussians)

        # Cal the WD loss and draw the distrb.
        # loss  = loss_fn_WD(pi, mu, sigma, target, N_gaussians)

        epoch_train_loss += loss.item()
        draw_loss(total_train_step, loss.item(),win_train_loss_str)

        # print("训练次数：{}，Loss：{}".format(total_train_step, loss.item()))

        # Optim
        optimizer.zero_grad()
        loss.backward()

        ########### before step() ###############

        # for name, parms in mlp.named_parameters():
        #     print("=====before step()=====")
        #     print('-->name:', name)
        #     print('-->params:', parms.data)
        #     # print('-->grad_requirs:',parms.requires_grad)
        #     print('-->grad_value:',parms.grad)
        #     print("===")

        ######

        # 反向传播时检测是否有异常值，定位code
        # with torch.autograd.detect_anomaly():
        #     loss.backward()

        ######
        # get_dot = register_hooks(loss)
        # dot = get_dot()
        # dot.save('tmp.dot')   # to get .dot
        # dot.render('tmp')     # to get pdf

        ######
        # Print grad check
        # v_n = []    # name
        # v_v = []    # value
        # v_g = []    # grad
        # for name, parameter in mlp.named_parameters():
        #     v_n.append(name)
        #     v_v.append(parameter.detach().cpu().numpy() if parameter is not None else [0])
        #     v_g.append(parameter.grad.detach().cpu().numpy() if parameter.grad is not None else [0])
        # for i in range(len(v_n)):
        #     # if np.max(v_v[i]).item() - np.min(v_v[i]).item() < 1e-6:
        #     #     color = bcolors.FAIL + '*'
        #     if np.isnan(v_g[i]).any() or np.isnan(v_v[i]).any():
        #         color = bcolors.FAIL + '*'
        #     else:
        #         color = bcolors.OKGREEN + ' '
        #     print('%svalue %s: %.3e ~ %.3e' % (color, v_n[i], np.min(v_v[i]).item(), np.max(v_v[i]).item()))
        #     print('%sgrad  %s: %.3e ~ %.3e' % (color, v_n[i], np.min(v_g[i]).item(), np.max(v_g[i]).item()))

        ######
        optimizer.step()

        # for name, parms in mlp.named_parameters():
        #     print("=====After step()=====")
        #     print('-->name:', name)
        #     print('-->params:', parms.data)
        #     # print('-->grad_requirs:',parms.requires_grad)
        #     print('-->grad_value:',parms.grad)
        #     print("===")

        # Only draw the 1st result in a training batch (5 in total)
        # if total_train_step % 20 == 0:
        #     with torch.no_grad():
        #         draw_mdn_2(pi[0,:].clone().detach(), mu[0,:].clone().detach(), sigma[0,:].clone().detach(), target[0].clone().detach(), input_data[0].clone().detach(),total_train_step, N_gaussians)
        #         # draw_mdn_cdf(pi[0,:].detach(), m[0], target[0].detach(), total_train_step, loss_list[0],N_gaussians)
        total_train_step += 1

    ########### Do validation
    mlp.eval()
    total_vali_metric, GT_metric = validate(mlp,val_loader,len(val_idx),opt.N_gaussians, opt.MIN_LOSS,device)
    draw_metric(epoch, total_vali_metric.cpu(), GT_metric)
    writer.add_scalars("metric/"+opt.tag_str,{"pred":total_vali_metric.cpu(),
                                "GT-1":GT_metric[0,0],
                                "GT-2":GT_metric[0,1]},epoch)
    mlp.train()

    # Plot the loss in this EPOCH
    print(f"========== IN EPOCH {epoch} the total loss is {epoch_train_loss} ==========")
    print(f"========== IN EPOCH {epoch} the vali NLL loss is {total_vali_metric.detach().cpu().numpy()} ==========")
    print(f"========== IN EPOCH {epoch} the GT metric is {GT_metric.detach().cpu().numpy()} ==========")

    # # Save the net structure
    # net_file_name = "net_after_epoch"+str(epoch)+".pth"
    # net_path = net_root_path + net_file_name
    # torch.save(mlp.state_dict(), net_path)

    # Plot loss of this EPOCH
    draw_loss(epoch, epoch_train_loss,win_train_epoch_loss_str)
    # writer.add_scalars("Loss of EPOCH",{"Train":epoch_train_loss},epoch)
    scheduler.step()

    # Record the params
    # for name,param in params:
    #     #参数的梯度
    #     #print("name:",name)
    #     #print("param.grad:",param.grad)
    #     # writer.add_histogram(tag = name +'_grad of EPOCH '+str(epoch),values=param.grad,global_step=epoch)
    #     # #参数值
    #     writer.add_histogram(tag = name +'data of EPOCH '+str(epoch),values=param.data,global_step=epoch)

# f.close()
train_end_time = time.time()
print(f"Total training time when epoch= *{EPOCH_NUM}* is *{train_end_time-train_start_time} *s")
writer.close()
print("Done")

========== Now this is EPOCH 0 ==========


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

- save

In [ ]:
model_path_MLE = "mlp_train=300_MLE.pth"
# model_path_WD = "mlp_train=300_WD_epoch=80.pth"
model_path_WD = "mlp_train=300_WD.pth"
model_path_LSE = "mlp_train=300_LSE.pth"
torch.save(mlp.state_dict(), model_path_LSE)
# torch.save(mlp.state_dict(), model_path_MLE)

# viz.delete_env("001_test")
